<a href="https://colab.research.google.com/github/Zishan-Shao/s-step-elasticnet-cd/blob/main/s_step_logistic_cd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/s-step-Logistic-Coordinate-Descent/data

/content/drive/My Drive/s-step-Logistic-Coordinate-Descent/data


$\forall l\in \{1,\ldots,s\}$

$\Delta \beta_j^{k+l} = \frac{y^T X e_j^{k+l} - e_j^{k+l} X^T \text{sig}(X\beta^k) - \sum_{i=1}^{l-1} e_j^{k+l} X^T \text{sig}(X\beta^k) \odot (1 - \text{sig}(X\beta^k)) \odot X e_j^{k+i} \Delta \beta_j^{k+i}}
{e_j^{k+l} X^T \text{sig}(X\beta^k) \odot (1 - \text{sig}(X\beta^k)) \odot X e_j^{k+l}},$

where $sig(x)$ is a sigmoid function.

$\textbf{Algorithm:}$ \\
选定一次交流是s.


sample s features from p features, each correspond to a basic base($e_{j}^{k+l}$ 表示第$l$个抽取的feature位置为1其余为0的基).

$
1_s = (e_j^k, \ldots, e_j^{k+s}) \in \mathbb{R}^{p \times (s+1)} \\
X 1_s = (X e_j^k, \ldots, X e_j^{k+s}) \in \mathbb{R}^{n \times (s+1)} \\
\text{sig}(X\beta^k) \in \mathbb{R}^{n \times 1}$

$
W = \text{sig}(X\beta^k) \odot (1 - \text{sig}(X\beta^k)) \in \mathbb{R}^{n \times 1} \\
$
$
A = (X 1_s)^T \text{sig}(X\beta^k)_{(s+1) \times 1} \quad \Rightarrow \quad A_{l} = e_j^{k+l} X^T \text{sig}(X\beta^k)
$
$
B = (X 1_s)^T (W \odot (X 1_s)) \in \mathbb{R}^{(s+1) \times (s+1)} \quad
$

具体来说$B$中, $W \odot (X 1_s)$需要把$W$重复(广播)为$(W,W,\dots,W)_{n\times s}$

$\Rightarrow B_{l,i} = e_j^{k+l} X^T \text{sig}(X\beta^k) \odot (1 - \text{sig}(X\beta^k)) \odot X e_j^{k+i}
$

for $l$ in $s$:

$\quad
\Delta \beta_j^{k+l} = \frac{y^T X e_j^{k+l} - A_{l} - \sum_{i=0}^{l-1} B_{l,i} \Delta \beta_j^{k+i}}{B_{l,l}}
$

$\beta_j^{k+s} = \beta_{j}^{k}+\sum_{i=0}^{s-1}\Delta \beta_j^{k+i}$

这个部分很重要

In [87]:
import numpy as np
from sklearn.datasets import make_classification
import pandas as pd
from scipy.special import expit  # sigmoid
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import time
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def logistic_gradient_descent(X, y, beta, beta_0, s, max_iter=2000, tol= 1e-4, lambda_para=0.2, alpha_para = 0.5):
    """
    Logistic Gradient Descent Algorithm with s-step updates.

    Parameters:
    X : np.ndarray
        Feature matrix (n x p).
    y : np.ndarray
        Target vector (n x 1).
    beta : np.ndarray
        Initial coefficients (p x 1).
    s : int
        Number of features to sample and update at each step.
    max_iter : int
        Maximum number of iterations.
    tol : float
        Tolerance for convergence.

    Returns:
    beta : np.ndarray
        Optimized coefficients.
    """
    # n = 100
    # p = 10
    # s = 5
    n, p = X.shape
    epsilon = 1e-8
    iter = 0
    # 每一轮其实是循环大致p/s次
    for iteration in range(max_iter*round(p/s + 1)):
        # # Sample s features from p features
        # sampled_indices = np.random.choice(p, s, replace=True)
        start = (iteration * s) % p
        end = start + s
        # print(end)
        flag = 0

        # 根据范围选择特征
        if end < p:
            flag = 0
            sampled_indices = np.arange(start, end)  # 在范围内直接取
        elif end == p: # this implies that the iteration p has been done
            flag = 1
            iter += 1
            print(iter)
            sampled_indices = np.arange(start, end)  # 在范围内直接取
        else:
            # 超出范围时取余（循环选择）
            flag = 1
            iter += 1
            sampled_indices = np.concatenate((np.arange(start, p), np.arange(0, end % p)))
        # print(sampled_indices)
        # Construct 1_s and X_{1_s}
        one_s = np.eye(p)[:, sampled_indices]  # (p x s)
        #print(one_s)

        X_1s = X @ one_s  # (n x s)
        # print(X_1s)

        #if iteration == 0:
        #   beta_old = 0  # (n x 1)
        # 初始化beta_old
        if iteration == 0:
          beta_old = beta.copy()

        if flag == 1:
        # 说明已经循环完一轮了
          # print(flag)
          print(np.linalg.norm(beta_old - beta))
          if np.linalg.norm(beta_old - beta) < tol*np.sqrt(p):
            print(f"ending iteration:{iter}")
            break
          # update beta_old
          beta_old = beta.copy()

        # Compute sigmoid
        sigmoid_X_beta = expit(beta_0 + X @ beta)  # (n x 1)
        # print("beta_0:", beta_0)
        # print("sigmoid_X_beta",sigmoid_X_beta)
        # # Check convergence
        # if np.linalg.norm(beta - beta_old) < tol and iteration > 2:
        #     print(f"Converged in {iteration + 1} iterations.")
        #     break


        # Compute W
        W = sigmoid_X_beta * (1 - sigmoid_X_beta)  # (n x 1)
        # print(W.shape)
        # print("W",W)
        delta_beta_0 = (np.sum(y)-np.sum(sigmoid_X_beta))/(np.sum(W)+epsilon)

        # if np.abs(delta_beta_0)<= np.abs(beta_0):

        beta_0 += delta_beta_0

        # Compute A
        A = (X_1s.T @ sigmoid_X_beta)  # (s x 1)
        #print(A.shape)

        # Compute B
        B = (X_1s.T @ (np.tile(W, (1, s)) * X_1s))  # (s x s)
        #print(B.shape)

        # Update beta for each feature in s
        delta_beta = np.zeros((p, 1))
        delta_beta_elastic = np.zeros((p, 1))
        for l in range(s):
            B_l_l = B[l, l]
            A_l = A[l]

            summation_term = 0
            for i in range(l):
                B_l_i = B[l, i]
                summation_term += B_l_i * delta_beta_elastic[sampled_indices[i]] #这个地方改了
            #print(summation_term)


            # 弹性网
            delta_beta_l = ((1/n)*(y.T @ X_1s[:, l] - A_l - summation_term)-lambda_para*(1-alpha_para)*beta[sampled_indices[l]]) / ((1/n)*B_l_l + lambda_para*(1-alpha_para)+epsilon)
            # delta_beta_l = ((1/n)*(y.T @ X_1s[:, l] - A_l - summation_term)-lambda_para*beta[sampled_indices[l]]) / ((1/n)*B_l_l + lambda_para)

            #print(f"delta_beta_l:{delta_beta_l}")
            # delta_beta[sampled_indices[l]] = delta_beta_l
            # beta[sampled_indices[l]] += delta_beta_l

            # 弹性网
            # print(delta_beta_l)
            # print(lambda_para*alpha_para/ ((1/n)*B_l_l + lambda_para*(1-alpha_para)))
            # print("*"*80)
            l1_term = lambda_para*alpha_para/ ((1/n)*B_l_l + lambda_para*(1-alpha_para)+epsilon)
            # if delta_beta_l < 0:
            #     print(f"less than 0{delta_beta_l}")
            #     print(f"{beta[sampled_indices[l]]}")
            #     print(f"l1_term:{l1_term}")
            delta_beta_elastic_l = 0

            if np.abs(beta[sampled_indices[l]] + delta_beta_l) <= l1_term:
                delta_beta_elastic_l = beta[sampled_indices[l]]
                beta[sampled_indices[l]] = 0
                # print(f"type 1:{beta[sampled_indices[l]]}")
            elif beta[sampled_indices[l]] + delta_beta_l < 0:
                delta_beta_elastic_l = delta_beta_l + l1_term
                beta[sampled_indices[l]] += delta_beta_elastic_l

                # print(f"type 2:{beta[sampled_indices[l]]}")
            else:
                delta_beta_elastic_l = delta_beta_l - l1_term
                beta[sampled_indices[l]] += delta_beta_elastic_l
               # print(f"type 3:{beta[sampled_indices[l]]}")
            # print(delta_beta_elastic_l )
            delta_beta_elastic[sampled_indices[l]] = delta_beta_elastic_l


        #print(beta)
        # # Check convergence
        # if np.linalg.norm(delta_beta_elastic) < tol:
        #     print(f"Converged in {iteration + 1} iterations.")
        #     print(np.count_nonzero(delta_beta_elastic))

        #     break

    return beta, beta_0


读数据

In [64]:
import pandas as pd
import numpy as np

# 定义文件路径
file_path = "/content/drive/My Drive/s-step-Logistic-Coordinate-Descent/data/colon-cancer.txt"

# 初始化存储
X = []
y = []

# 解析文件内容d
with open(file_path, 'r') as f:
    for line in f:
        # 分割每一行数据
        parts = line.strip().split()
        # 提取目标变量 y（第一列），将浮点数转换为整数
        y.append(int(float(parts[0])))
        # 提取特征并存入字典，键为特征索引，值为特征值
        features = {int(kv.split(':')[0]): float(kv.split(':')[1]) for kv in parts[1:]}
        X.append(features)

# 转换为 Pandas DataFrame，填充缺失值为 0
X = pd.DataFrame(X).fillna(0).sort_index(axis=1)  # 按列排序

# 将列名转换为字符串（保持一致性）
X.columns = X.columns.astype(str)

# 转换目标变量为 NumPy 数组
y = np.array(y)

# 将目标变量从 -1 和 1 转换为 0 和 1
y = np.where(y == -1, 0, y)

# 检查结果
print("Feature matrix X (first 5 rows):")
print(X.head())
print("\nTarget variable y (first 5 values):")
print(y[:5])

# 保存特征矩阵 X 和目标变量 y 到文件（如果需要）
# X.to_csv("X.csv", index=False)
# np.savetxt("y.csv", y, delimiter=",", fmt="%d")

# 完成
print("\nData processing complete.")


Feature matrix X (first 5 rows):
          1         2         3         4         5         6         7  \
0  2.080750  1.099070  0.927763  1.029080 -0.130763  1.265460 -0.436286   
1  1.109460  0.786453  0.445560 -0.146323 -0.996316  0.555759  0.290734   
2 -0.676530  1.693100  1.559250  1.559980 -0.982179 -1.358510 -1.313990   
3  0.534396  1.677540  1.489030  0.778605 -0.183776 -1.116850 -1.487560   
4 -1.018900  0.511080  0.755641  1.013820  0.529899  0.160440 -0.087055   

          8         9        10  ...      1991      1992      1993      1994  \
0  0.728881  2.107980  1.359870  ... -0.825403 -0.138451  0.382957  0.876697   
1 -0.145259  1.132660  0.559093  ... -1.056290 -0.205499 -1.815370  0.324373   
2 -0.455067  0.295214  0.290694  ...  1.242970  1.230160 -2.039000  2.366090   
3 -0.579511  0.292683  1.345480  ...  0.559852 -0.593149 -4.440580  1.720710   
4  1.295290  0.458736  0.714082  ...  0.227110  0.497628 -0.083921 -0.382733   

       1995      1996      1997    

In [88]:

# Example usage
np.random.seed(42)

# Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 假设 X 是特征数据，y 是目标变量
# 确保 X 和 y 的顺序一致
n_samples = len(X)
split_index = int(n_samples * 0.8)  # 按 80% 划分

# 按顺序切分数据
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# # Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
n, p = X.shape
beta_init = np.zeros((p, 1))
beta_0_init = np.log(np.mean(y_train) / (1 - np.mean(y_train)))
s = 1024


# Measure time for coordinate gradient descent
start_time_cg = time.time()
beta, beta_0 = logistic_gradient_descent(X_train, y_train, beta_init,beta_0_init, s)
y_pred_cg = expit(beta_0 + X_test @ beta) >= 0.5
accuracy_cg = accuracy_score(y_test, y_pred_cg)
cg_time = time.time() - start_time_cg
print(f"Coordinate Gradient Descent beta:{beta_0},{beta}")
print(f"Coordinate Gradient Descent Accuracy: {accuracy_cg:.4f}")
print(f"Coordinate Gradient Descent Time: {cg_time:.4f} seconds")


# # Measure time for coordinate gradient descent
# start_time_cg = time.time()
# beta, beta_0 = logistic_gradient_descent(X_train, y_train, beta_init,beta_0_init, s=1)
# y_pred_cg = expit(beta_0 + X_test @ beta) >= 0.5
# accuracy_cg = accuracy_score(y_test, y_pred_cg)
# cg_time = time.time() - start_time_cg
# #print(f"Coordinate Gradient Descent bera:{beta}")
# print(f"Coordinate Gradient Descent Accuracy: {accuracy_cg:.4f}")
# print(f"Coordinate Gradient Descent Time: {cg_time:.4f} seconds")
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import time

# 定义超参数
alpha = 0.2
l1_ratio = 0.5

# 开始计时
start_time_en = time.time()

# 创建 ElasticNet 模型
elastic_net_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=2000, random_state=42)
elastic_net_model.fit(X_train, y_train)

# 进行预测
y_pred_en = elastic_net_model.predict(X_test) >= 0.5

# 计算准确率
accuracy_en = accuracy_score(y_test, y_pred_en)
print(f"ElasticNet Accuracy: {accuracy_en:.4f}")

# 计算性能指标
mse = mean_squared_error(y_test, y_pred_en)
r2 = r2_score(y_test, y_pred_en)
en_time = time.time() - start_time_en

# 打印结果
accuracy_cgd = accuracy_score(y_test, y_pred_en)

print(f"Coordinate Gradient Descent Accuracy: {accuracy_cgd:.4f}")
print(f"ElasticNet Mean Squared Error: {mse:.4f}")
print(f"ElasticNet R2 Score: {r2:.4f}")
print(f"ElasticNet Time: {en_time:.4f} seconds")

# Measure time for Logistic Regression (default solver)
start_time_lr = time.time()
logistic_model = LogisticRegression(max_iter=2000, random_state=42)
logistic_model.fit(X_train, y_train)
y_pred_lr = logistic_model.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
lr_time = time.time() - start_time_lr
print(f"Logistic Regression Accuracy: {accuracy_lr:.4f}")
print(f"Logistic Regression Time: {lr_time:.4f} seconds")

# Measure time for Logistic Regression with liblinear solver
start_time_lr_liblinear = time.time()
# # # L1 正则化 (lambda=0.1)
# logistic_model_liblinear = LogisticRegression(
#     solver='liblinear',
#     penalty='l1',  # 或 'l2'，根据需要选择
#     C=2,          # 设置 lambda 的倒数
#     max_iter=2000,
#     random_state=42
# )
# logistic_model_liblinear = LogisticRegression(
#     solver='saga',
#     penalty='elasticnet',  # 指定 Elastic Net 正则化
#     l1_ratio=0.5,          # 设置 alpha 参数
#     C=5,                  # 设置 lambda 的倒数
#     max_iter=2000,
#     random_state=42
# )
logistic_model_liblinear = LogisticRegression(solver='liblinear', max_iter=2000, random_state=42)
logistic_model_liblinear.fit(X_train, y_train)
y_pred_lr_liblinear = logistic_model_liblinear.predict(X_test)
accuracy_lr_liblinear = accuracy_score(y_test, y_pred_lr_liblinear)
lr_liblinear_time = time.time() - start_time_lr_liblinear
print(f"Logistic Regression (liblinear) Accuracy: {accuracy_lr_liblinear:.4f}")
print(f"Logistic Regression (liblinear) Time: {lr_liblinear_time:.4f} seconds")


0.7918425545197283
0.6588422017577934
0.3951664546316623
0.3285337046778384
0.29493786611430534
0.2122478788966817
0.17015227429762877
0.11001601738650993
0.09945463009945861
0.09189758938218505
0.06361961352770681
0.05955981783943549
0.052042884871088026
0.058690730856820826
0.050244410643121074
0.04695798156752664
0.04101909830864024
0.03633168875594555
0.032441148699122964
0.029012471373662647
0.024912464653002987
0.017871837723092848
0.021336843243015897
0.019143479995714422
0.016337904986377095
0.015934316012337187
0.014680647430076806
0.013156665599788472
0.011170403005435859
0.010015479368120394
0.009090837147805569
0.008158886754348012
0.007199916418502313
0.005993517390465304
0.0050861099643707015
0.006263433201963035
0.00419496069160578
ending iteration:37
Coordinate Gradient Descent beta:-1.0394172315260441,[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Coordinate Gradient Descent Accuracy: 0.6923
Coordinate Gradient Descent Time: 174.8882 seconds
ElasticNet Accuracy: 0.6923
Coor

We do not use python's glmnet as baseline because it is not accurate, so we implement a python-based

Adaptive Learning rate: with closer to the solution, we make the learning rate smaller so less likely to fall into

# Code Optimization & Baseline

### Part 1: s-step code optimization

first one is with sparsification, but not well on colon-cancer dataset, which is dense

In [ ]:
import numpy as np
import pandas as pd
import time

from scipy.sparse import csr_matrix, diags
from scipy.special import expit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def logistic_gradient_descent_sparse(X, y, beta, beta_0, s, max_iter=2000, tol=1e-4,
                                     lambda_para=0.2, alpha_para=0.5):
    """
    Logistic Gradient Descent Algorithm with s-step updates (sparsified).

    Parameters:
    -----------
    X : np.ndarray or scipy.sparse.csr_matrix
        Feature matrix (n x p).
    y : np.ndarray
        Target vector (n,). Values should be 0 or 1.
    beta : np.ndarray
        Initial coefficients of shape (p, 1) or (p,).
    beta_0 : float
        Initial intercept term.
    s : int
        Number of features to sample and update at each step.
    max_iter : int
        Maximum number of iterations (each iteration updates `s` features).
    tol : float
        Tolerance for convergence based on L2 norm of coefficient change.
    lambda_para : float
        Regularization strength for elastic net.
    alpha_para : float
        Mixing parameter for elastic net. alpha=1 => L1, alpha=0 => L2.

    Returns:
    --------
    beta : np.ndarray
        Optimized coefficients (shape same as input).
    beta_0 : float
        Optimized intercept term.
    """
    # Ensure X is in csr_matrix format
    if not isinstance(X, csr_matrix):
        X = csr_matrix(X)

    n, p = X.shape
    epsilon = 1e-8
    iteration_count = 0

    # If beta is (p,1), flatten it to (p,) for easier indexing
    # but keep in mind we might need to reshape back later
    if beta.ndim == 2 and beta.shape[1] == 1:
        beta = beta.flatten()

    beta_old = beta.copy()

    # Number of outer loops: each iteration updates 's' features, so we cycle ~p/s times in one pass
    max_loops = max_iter * (round(p / s + 1))

    for iteration in range(max_loops):

        # Determine sampled indices
        start = (iteration * s) % p
        end = start + s
        if end <= p:
            sampled_indices = np.arange(start, end)
        else:
            # Wrap around
            sampled_indices = np.concatenate(
                (np.arange(start, p), np.arange(0, end % p))
            )

        # Construct X_1s for selected features
        # Create a sparse identity-like submatrix with ones on [sampled_indices, range(s)]
        one_s = csr_matrix(
            (np.ones(len(sampled_indices)),
             (sampled_indices, np.arange(len(sampled_indices)))),
            shape=(p, len(sampled_indices))
        )
        X_1s = X @ one_s  # shape (n, s)

        # Check for convergence each time we hit a multiple of p features
        # i.e., after ~one full pass of p features
        if end % p == 0 and iteration != 0:
            iteration_count += 1
            #print(f"Epoch {iteration_count}: loss {np.linalg.norm(beta_old - beta)}.")
            #print(np.linalg.norm(beta_old - beta))
            diff_norm = np.linalg.norm(beta_old - beta)
            if diff_norm < tol * np.sqrt(p):
                print(f"Converged at iteration {iteration_count}")
                break
            beta_old = beta.copy()



        # Compute sigmoid
        # X @ beta => shape (n,). Make sure shapes align.
        logits = beta_0 + (X @ beta)
        sigmoid_X_beta = expit(logits)

        # Compute W diagonal (as a 1D array)
        W = (sigmoid_X_beta * (1 - sigmoid_X_beta)).flatten()

        # Construct diagonal weight matrix in sparse form
        W_sparse = diags(W, 0, shape=(n, n), format='csr')

        # Update beta_0
        # sum of residuals over sum of W
        delta_beta_0 = (np.sum(y) - np.sum(sigmoid_X_beta)) / (np.sum(W) + epsilon)
        beta_0 += delta_beta_0

        # Compute A and B
        # A = X_1s^T * sigmoid_X_beta
        A = X_1s.T @ sigmoid_X_beta  # shape (s,)
        # B = X_1s^T * W_sparse * X_1s
        B = X_1s.T @ (W_sparse @ X_1s)  # shape (s,s)

        # Update beta for sampled features
        for l in range(len(sampled_indices)):
            idx = sampled_indices[l]

            B_l_l = B[l, l]
            A_l = A[l]

            # summation_term = B[l, :l] @ beta for indices in sampled_indices[:l]
            # Safely convert partial row to dense if needed
            B_l_range = B[l, :l]
            if hasattr(B_l_range, "toarray"):
                B_l_range = B_l_range.toarray().ravel()  # flatten to 1D

            summation_term = B_l_range @ beta[sampled_indices[:l]]

            # Numerator for coordinate descent
            numerator = ((1.0 / n) * (y @ X_1s[:, l] - A_l - summation_term)
                         - lambda_para * (1 - alpha_para) * beta[idx])

            denominator = ((1.0 / n) * B_l_l
                           + lambda_para * (1 - alpha_para)
                           + epsilon)

            delta_beta_l = numerator / denominator

            # L1 shrinkage (soft thresholding)
            l1_term = lambda_para * alpha_para / denominator
            current_beta = beta[idx] + delta_beta_l

            #if np.abs(current_beta) <= l1_term:
            #    beta[idx] = 0.0
            #elif current_beta < 0:
            #    beta[idx] = current_beta + l1_term
            #else:
            #    beta[idx] = current_beta - l1_term
            if np.abs(current_beta) <= l1_term:
                beta[idx] = 0.0
            elif current_beta < 0:
                beta[idx] = float(current_beta + l1_term)  # or .item()
            else:
                beta[idx] = float(current_beta - l1_term)  # or .item()


    # Reshape beta back to (p,1) if needed
    if beta.ndim == 1:
        beta = beta.reshape(-1, 1)

    return beta, beta_0


# ==============================================================================
#                         TEST SCRIPT BELOW
# ==============================================================================

# 1. Data Loading and Preprocessing
file_path = "/content/drive/My Drive/s-step-Logistic-Coordinate-Descent/data/colon-cancer.txt"
X_list, y_list = [], []

with open(file_path, 'r') as f:
    for line in f:
        parts = line.strip().split()
        # Convert label -1 to 0
        label = int(float(parts[0]))
        y_list.append(0 if label == -1 else label)
        features = {
            int(kv.split(':')[0]): float(kv.split(':')[1])
            for kv in parts[1:]
        }
        X_list.append(features)

X = pd.DataFrame(X_list).fillna(0).sort_index(axis=1)  # shape (n, p)
y = np.array(y_list)  # shape (n,)

print('Data Loading Complete')

# 2. Train / Test Split
n_samples = len(X)
split_index = int(n_samples * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# 3. Standardize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert to Sparse (optional, beneficial if data is truly sparse)
X_train_csr = csr_matrix(X_train)
X_test_csr = csr_matrix(X_test)

# 5. Initialize parameters for logistic gradient descent
n, p = X.shape
beta_init = np.zeros((p, 1))
beta_0_init = np.log(np.mean(y_train) / (1 - np.mean(y_train) + 1e-12))  # avoid zero
s = 1024  # number of features updated at once

# 6. Run Custom Logistic Gradient Descent
start_time_cg = time.time()
beta, beta_0 = logistic_gradient_descent_sparse(
    X_train_csr,
    y_train,
    beta_init,
    beta_0_init,
    s,
    max_iter=2000,
    tol=1e-4,
    lambda_para=0.2,
    alpha_para=0.5
)
cg_time = time.time() - start_time_cg

# 7. Evaluate Performance
# beta should be shape (p,1). Do X_test @ beta => shape (n_test,)
logits_test = beta_0 + X_test_csr @ beta
y_pred_cg = expit(logits_test).ravel() >= 0.5
accuracy_cg = accuracy_score(y_test, y_pred_cg)

print(f"Custom Gradient Descent Accuracy: {accuracy_cg:.4f}, Time: {cg_time:.4f}s")

# 8. Compare with scikit-learn’s LogisticRegression
start_time_lr = time.time()
logistic_model = LogisticRegression(max_iter=2000, random_state=42)
logistic_model.fit(X_train_csr, y_train)
lr_time = time.time() - start_time_lr

y_pred_lr = logistic_model.predict(X_test_csr)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

print(f"Logistic Regression (default solver) Accuracy: {accuracy_lr:.4f}, Time: {lr_time:.4f}s")

start_time_lr_liblinear = time.time()
logistic_model_liblinear = LogisticRegression(solver='liblinear', max_iter=2000, random_state=42)
logistic_model_liblinear.fit(X_train_csr, y_train)
lr_liblinear_time = time.time() - start_time_lr_liblinear

y_pred_lr_liblinear = logistic_model_liblinear.predict(X_test_csr)
accuracy_lr_liblinear = accuracy_score(y_test, y_pred_lr_liblinear)

print(f"Logistic Regression (liblinear) Accuracy: {accuracy_lr_liblinear:.4f}, Time: {lr_liblinear_time:.4f}s")

# Final summary
print("\n--- Summary ---")
print(f"Custom Logistic CD Accuracy: {accuracy_cg:.4f} | Time: {cg_time:.4f}s")
print(f"Logistic Regression (default) Accuracy: {accuracy_lr:.4f} | Time: {lr_time:.4f}s")
print(f"Logistic Regression (liblinear) Accuracy: {accuracy_lr_liblinear:.4f} | Time: {lr_liblinear_time:.4f}s")


In [102]:
import numpy as np
import pandas as pd
import time

from scipy.special import expit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

###############################################################################
# Vectorized Logistic Coordinate Descent Function
###############################################################################
def logistic_cd_fast(
    X, y, beta, beta_0, s, max_iter=2000, tol=1e-4,
    lambda_para=0.2, alpha_para=0.5
):
    """
    Logistic Coordinate Descent with s-step block updates (elastic net regularization).

    Parameters
    ----------
    X : np.ndarray of shape (n, p)
        Feature matrix.
    y : np.ndarray of shape (n,)
        Binary target vector (0/1).
    beta : np.ndarray of shape (p, 1) or (p,)
        Initial coefficients.
    beta_0 : float
        Initial intercept.
    s : int
        Number of features to update at a time.
    max_iter : int
        Maximum number of epochs (full passes).
    tol : float
        Convergence threshold based on L2 norm of coefficient change.
    lambda_para : float
        Regularization strength for elastic net.
    alpha_para : float
        Mixing parameter for elastic net (0 = L2, 1 = L1).

    Returns
    -------
    beta : np.ndarray of shape (p, 1)
        Learned coefficients.
    beta_0 : float
        Learned intercept.
    """

    # Flatten beta to shape (p,) for easier indexing
    if beta.ndim == 2 and beta.shape[1] == 1:
        beta = beta.flatten()

    n, p = X.shape
    epsilon = 1e-8
    epoch = 0
    beta_old = beta.copy()

    # We'll do max_iter passes, each pass ~ p/s block-updates
    total_iter = max_iter * int(np.ceil(p / s))

    # Helper to compute logistic loss w/ elastic net penalty
    def logistic_loss(y_true, y_prob, coef):
        eps = 1e-8
        # Negative log-likelihood
        neg_ll = -(
            y_true @ np.log(y_prob + eps) +
            (1 - y_true) @ np.log(1 - y_prob + eps)
        ) / n
        # Elastic net penalty
        l2_term = 0.5 * np.sum(coef**2)
        l1_term = np.sum(np.abs(coef))
        penalty = lambda_para * ((1 - alpha_para) * l2_term + alpha_para * l1_term)
        return neg_ll + penalty

    for iteration in range(total_iter):
        start = (iteration * s) % p
        end = start + s
        if end <= p:
            sampled_indices = np.arange(start, end)
        else:
            # Wrap around if needed
            sampled_indices = np.concatenate([
                np.arange(start, p),
                np.arange(0, end % p)
            ])

        # Check if we've completed one full pass of p features => an "epoch"
        if end % p == 0 and iteration != 0:
            epoch += 1
            # Compute current loss and print progress
            y_prob = expit(beta_0 + X @ beta)
            loss_val = logistic_loss(y, y_prob, beta)
            diff_norm = np.linalg.norm(beta_old - beta)
            print(f"Epoch {epoch}, Loss={loss_val:.6f}, ||Δβ||={diff_norm:.6f}")

            # Convergence check
            if diff_norm < tol * np.sqrt(p):
                print(f"Converged at epoch {epoch}")
                break
            beta_old = beta.copy()

        # Compute sigmoid & derivative
        sig = expit(beta_0 + X @ beta)
        W = sig * (1 - sig)

        # Update intercept
        delta_beta_0 = (np.sum(y) - np.sum(sig)) / (np.sum(W) + epsilon)
        beta_0 += delta_beta_0

        # Extract the block X_1s
        X_1s = X[:, sampled_indices]

        # Compute A and B for the chosen block
        A = X_1s.T @ sig
        B = X_1s.T @ (W[:, None] * X_1s)

        # Update coefficients in this block
        for l in range(len(sampled_indices)):
            idx = sampled_indices[l]
            B_ll = B[l, l]
            A_l = A[l]

            # Summation of updates from previously updated coords in this block
            block_indices = sampled_indices[:l]
            summation_term = B[l, :l] @ (beta[block_indices] - beta_old[block_indices])

            numerator = (
                (1.0 / n)*(y @ X_1s[:, l] - A_l - summation_term)
                - lambda_para * (1 - alpha_para) * beta[idx]
            )
            denominator = ((1.0 / n)*B_ll + lambda_para*(1 - alpha_para) + epsilon)
            db_l = numerator / denominator
            l1_term = lambda_para * alpha_para / denominator

            candidate = beta[idx] + db_l
            if abs(candidate) <= l1_term:
                beta[idx] = 0.0
            elif candidate < 0:
                beta[idx] = candidate + l1_term
            else:
                beta[idx] = candidate - l1_term

    # Reshape back to (p,1)
    return beta.reshape(-1, 1), beta_0

###############################################################################
# Test Script (Runs Directly)
###############################################################################

# 1. Load data (colon-cancer.txt, for example). Update path if needed:
file_path = "/content/drive/My Drive/s-step-Logistic-Coordinate-Descent/data/colon-cancer.txt"
X_list, y_list = [], []

with open(file_path, 'r') as f:
    for line in f:
        parts = line.strip().split()
        label = int(float(parts[0]))
        y_list.append(0 if label == -1 else label)
        features = {
            int(kv.split(':')[0]): float(kv.split(':')[1])
            for kv in parts[1:]
        }
        X_list.append(features)

X = pd.DataFrame(X_list).fillna(0).sort_index(axis=1)
y = np.array(y_list)
print("Data loaded. Shape of X:", X.shape)

# 2. Train/Test Split
n_samples = X.shape[0]
split_index = int(n_samples * 0.8)
X_train = X.iloc[:split_index].values
y_train = y[:split_index]
X_test  = X.iloc[split_index:].values
y_test  = y[split_index:]

# 3. Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Initialization
n, p = X_train.shape
beta_init = np.zeros((p, 1))
eps = 1e-12
y_mean = np.mean(y_train)
beta_0_init = np.log((y_mean + eps) / (1 - y_mean + eps))

# 5. Run our logistic CD
s = 1024
max_iter = 200
start_time_cd = time.time()
beta_cd, beta_0_cd = logistic_cd_fast(
    X_train, y_train, beta_init, beta_0_init, s,
    max_iter=max_iter, tol=1e-4, lambda_para=0.2, alpha_para=0.5
)
cd_time = time.time() - start_time_cd

# 6. Evaluate Performance
logits_test_cd = beta_0_cd + X_test @ beta_cd
y_prob_cd = expit(logits_test_cd).ravel()
y_pred_cd = (y_prob_cd >= 0.5).astype(int)
acc_cd = accuracy_score(y_test, y_pred_cd)

print(f"\nCustom Logistic CD => Accuracy: {acc_cd:.4f}, Time: {cd_time:.2f}s")

# 7. Compare with scikit-learn LogisticRegression
#lr_start = time.time()
#lr_model = LogisticRegression(max_iter=2000, random_state=42)
#lr_model.fit(X_train, y_train)
#lr_time = time.time() - lr_start

#y_pred_lr = lr_model.predict(X_test)
#acc_lr = accuracy_score(y_test, y_pred_lr)

#print(f"Sklearn LogisticRegression => Accuracy: {acc_lr:.4f}, Time: {lr_time:.2f}s")

print(beta_cd[1:10])
print(beta_0_cd)


Data loaded. Shape of X: (62, 2000)
Epoch 1, Loss=0.463430, ||Δβ||=0.394558
Epoch 2, Loss=0.440034, ||Δβ||=0.338407
Epoch 3, Loss=0.437141, ||Δβ||=0.376197

Custom Logistic CD => Accuracy: 0.6923, Time: 18.86s
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
-1.046029373064092


tolerance based stoppage

In [113]:
def logistic_cd_fast(
    X, y, beta, beta_0, s, max_iter=2000, tol=1e-4,
    lambda_para=0.2, alpha_para=0.5
):
    """
    Logistic Coordinate Descent with s-step block updates (elastic net regularization).
    """
    # Flatten beta to shape (p,) for easier indexing
    if beta.ndim == 2 and beta.shape[1] == 1:
        beta = beta.flatten()

    n, p = X.shape
    epsilon = 1e-8
    epoch = 0
    beta_old = beta.copy()

    # We'll do max_iter passes, each pass ~ p/s block-updates
    total_iter = max_iter * int(np.ceil(p / s))

    # Helper to compute logistic loss w/ elastic net penalty
    def logistic_loss(y_true, y_prob, coef):
        eps = 1e-8
        # Negative log-likelihood
        neg_ll = -(
            y_true @ np.log(y_prob + eps) +
            (1 - y_true) @ np.log(1 - y_prob + eps)
        ) / n
        # Elastic net penalty
        l2_term = 0.5 * np.sum(coef**2)
        l1_term = np.sum(np.abs(coef))
        penalty = lambda_para * ((1 - alpha_para) * l2_term + alpha_para * l1_term)
        return neg_ll + penalty

    # Coordinate Descent
    for iteration in range(total_iter):
        # Determine which block of coordinates to update
        start = (iteration * s) % p
        end = start + s
        if end <= p:
            sampled_indices = np.arange(start, end)
        else:
            # Wrap around if needed
            sampled_indices = np.concatenate([
                np.arange(start, p),
                np.arange(0, end % p)
            ])

        # Check if we've completed one full pass of p features => an "epoch"
        if end % p == 0 and iteration != 0:
            epoch += 1

            # Evaluate progress after each epoch
            y_prob = expit(beta_0 + X @ beta)
            loss_val = logistic_loss(y, y_prob, beta)
            diff_norm = np.linalg.norm(beta_old - beta)
            print(f"Epoch {epoch}, Loss={loss_val:.6f}, diff_beta={diff_norm:.6f}")

            # Check for convergence
            if diff_norm < tol * np.sqrt(p):
                print(f"Converged at epoch {epoch}")
                break

            # Update beta_old for the next epoch check
            beta_old = beta.copy()

        # ----- Update step below -----
        # Compute sigmoid & derivative
        sig = expit(beta_0 + X @ beta)
        W = sig * (1 - sig)

        # Update intercept
        delta_beta_0 = (np.sum(y) - np.sum(sig)) / (np.sum(W) + epsilon)
        beta_0 += delta_beta_0

        # Extract the block X_1s
        X_1s = X[:, sampled_indices]

        # Compute A and B for the chosen block
        A = X_1s.T @ sig
        B = X_1s.T @ (W[:, None] * X_1s)

        # Update coefficients in this block
        for l in range(len(sampled_indices)):
            idx = sampled_indices[l]
            B_ll = B[l, l]
            A_l = A[l]

            # Summation from previously updated coords in this block
            block_indices = sampled_indices[:l]
            summation_term = B[l, :l] @ (beta[block_indices] - beta_old[block_indices])

            numerator = (
                (1.0 / n) * (y @ X_1s[:, l] - A_l - summation_term)
                - lambda_para * (1 - alpha_para) * beta[idx]
            )
            denominator = ((1.0 / n) * B_ll + lambda_para * (1 - alpha_para) + epsilon)
            db_l = numerator / denominator
            l1_term = lambda_para * alpha_para / denominator

            candidate = beta[idx] + db_l
            if abs(candidate) <= l1_term:
                beta[idx] = 0.0
            elif candidate < 0:
                beta[idx] = candidate + l1_term
            else:
                beta[idx] = candidate - l1_term

    # Reshape back to (p,1)
    return beta.reshape(-1, 1), beta_0

In [114]:
###############################################################################
# Test Script (Runs Directly)
###############################################################################

# 1. Load data (colon-cancer.txt, for example). Update path if needed:
file_path = "/content/drive/My Drive/s-step-Logistic-Coordinate-Descent/data/colon-cancer.txt"
X_list, y_list = [], []

with open(file_path, 'r') as f:
    for line in f:
        parts = line.strip().split()
        label = int(float(parts[0]))
        y_list.append(0 if label == -1 else label)
        features = {
            int(kv.split(':')[0]): float(kv.split(':')[1])
            for kv in parts[1:]
        }
        X_list.append(features)

X = pd.DataFrame(X_list).fillna(0).sort_index(axis=1)
y = np.array(y_list)
print("Data loaded. Shape of X:", X.shape)

# 2. Train/Test Split
n_samples = X.shape[0]
split_index = int(n_samples * 0.8)
X_train = X.iloc[:split_index].values
y_train = y[:split_index]
X_test  = X.iloc[split_index:].values
y_test  = y[split_index:]

# 3. Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Initialization
n, p = X_train.shape
beta_init = np.zeros((p, 1))
eps = 1e-12
y_mean = np.mean(y_train)
beta_0_init = np.log((y_mean + eps) / (1 - y_mean + eps))

# 5. Run our logistic CD
s = 1024
max_iter = 2000
start_time_cd = time.time()
beta_cd, beta_0_cd = logistic_cd_fast(
    X_train, y_train, beta_init, beta_0_init, s,
    max_iter=max_iter, tol=1e-4, lambda_para=0.2, alpha_para=0.5
)
cd_time = time.time() - start_time_cd

# 6. Evaluate Performance
logits_test_cd = beta_0_cd + X_test @ beta_cd
y_prob_cd = expit(logits_test_cd).ravel()
y_pred_cd = (y_prob_cd >= 0.5).astype(int)
acc_cd = accuracy_score(y_test, y_pred_cd)

print(f"\nCustom Logistic CD => Accuracy: {acc_cd:.4f}, Time: {cd_time:.2f}s")


print(beta_cd[1:10])
print(beta_0_cd)


Data loaded. Shape of X: (62, 2000)
Epoch 1, Loss=0.463430, diff_beta=0.394558
Epoch 2, Loss=0.440034, diff_beta=0.338407
Epoch 3, Loss=0.437141, diff_beta=0.376197
Epoch 4, Loss=0.432828, diff_beta=0.373986
Epoch 5, Loss=0.431867, diff_beta=0.368230
Epoch 6, Loss=0.429125, diff_beta=0.327126
Epoch 7, Loss=0.428157, diff_beta=0.235275
Epoch 8, Loss=0.428039, diff_beta=0.212990
Epoch 9, Loss=0.427304, diff_beta=0.175854
Epoch 10, Loss=0.427604, diff_beta=0.180247
Epoch 11, Loss=0.427750, diff_beta=0.156974
Epoch 12, Loss=0.428478, diff_beta=0.159803
Epoch 13, Loss=0.427320, diff_beta=0.117853
Epoch 14, Loss=0.425337, diff_beta=0.111355
Epoch 15, Loss=0.424723, diff_beta=0.090813
Epoch 16, Loss=0.424619, diff_beta=0.072253
Epoch 17, Loss=0.424634, diff_beta=0.062932
Epoch 18, Loss=0.424661, diff_beta=0.058913
Epoch 19, Loss=0.424551, diff_beta=0.047309
Epoch 20, Loss=0.424399, diff_beta=0.041513
Epoch 21, Loss=0.424297, diff_beta=0.032220
Epoch 22, Loss=0.424182, diff_beta=0.028811
Epoch

Distributed Acceleration